In [2]:
import numpy as np 
import pandas as pd 

- a common workflow for model development is to use pandas for data loading and cleaning before switching over to a modeling library to build the model itself.
- an important part of the model development process is called "feature engineering" in machine learning.
- this can describe any data transformation or analytics that extract information from a raw dataset that may be useful in a modeling context.
- the data aggregation and GroupBy tools we have explored are used often in a feature engineering context.

In [3]:
# [ While details of "good" feature engineering are out of scope for this book, I will show some methods to make switching between data manipulation with pandas and modeling as painless as possible ]

# the point of contact between pandas and other analysis libraries is usually NumPy arrays.

# to turn a DataFrame into a NumPy array, use the to_numpy method
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [4]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [5]:
data.to_numpy()

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [6]:
# to convert back, pass a two-dim ndarray with optional column names
df2 = pd.DataFrame(data.to_numpy(), columns=['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [8]:
# the to_numpy method is intended to be used when your data is homogeneous
# if you have heterogeneous data, the result will be an ndarray of Python objects

df3 = data.copy()

In [9]:
df3['strings'] = ['a', 'b', 'c', 'd', 'e']
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [11]:
df3.to_numpy()

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [12]:
# for some models, you may wish to use only a subset of the columns
# recommended: use loc indexing with to_numpy

model_cols = ['x0', 'x1']
data.loc[:, model_cols].to_numpy()

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

- Some libraries have native support for pandas and do some of this work for you automatically: converting to NumPy from DataFrame and attaching model parameter names to the columns of output tables or Series. 
- In other cases, you will have to perform this “metadata management” manually.

In [13]:
# look at panda's Categorical type and the pandas.get_dummies function
# suppose we had a nonnumeric column in our example dataset

data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [14]:
# if we want to replace the 'category' column with dummy variabes, we create dummy variables, drop the 'category' column, and then join the result

dummies = pd.get_dummies(data.category, prefix="category")

data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,True,False
1,2,-0.01,0.0,False,True
2,3,0.25,3.6,True,False
3,4,-4.10,1.3,True,False
4,5,0.00,-2.0,False,True


- there are some nuances to fitting certain statistical models with dummy variables.
- It may be simpler and less error-prone to use Patsy when you have more than simple numeric columns.

## [ Creating Model Descriptions with Patsy ]
- Patsy is a Python library for describing statistical models (especially linear modesl) with a string-based "formula syntax", which is inspired by (but not exactly the same as) the formula syntax used by the R and S statistical programming languages.

- It is installed automatically when you install statsmodels.
- Patsy is well supported for specifying linear models in statsmodels.
- Patsy's formulas are a special string syntax that looks like: y ~ x0 + x1
- The syntax a + b does not mean to add a and b, but rather that these are terms in the design matrix created for the model.
- The patsy.dmatrices function takes a formula string along with a dataset (which can be a DataFrame or a dictionary of arrays) and produces design matrices for a linear model.

In [15]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [17]:
import patsy
y, X = patsy.dmatrices('y ~ x0 + x1', data)

# now we have 
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [18]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [19]:
# these patsy designmatrix instances are NumPy ndarrays with additional metadata
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [20]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [21]:
# where the Intercept term came from ?
# this is a convention for linear models like ordinary least squares (OLS) regression.
# we can suppress the intercept by adding the term + 0 to the model

patsy.dmatrices('y ~ x0 + x1 + 0', data)

(DesignMatrix with shape (5, 1)
      y
   -1.5
    0.0
    3.6
    1.3
   -2.0
   Terms:
     'y' (column 0),
 DesignMatrix with shape (5, 2)
   x0     x1
    1   0.01
    2  -0.01
    3   0.25
    4  -4.10
    5   0.00
   Terms:
     'x0' (column 0)
     'x1' (column 1))

In [22]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [23]:
# the patsy objects can be passed directly into algorithms like numpy.linalg.lstsq, which performs an ordinary least squares regression
coef, resid, _, _ = np.linalg.lstsq(X, y)

# the model metadata is retained in the design_info attribute, so you can reattach the model column names to the fitted coefficients to obtain a Series
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [24]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

## [ Data Transformations in Patsy Formulas ]
you can mix python code into your Patsy formulas; when evaluating the formula, the library will try to find the functions you use in the enclosing scope

In [25]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

- As part of a modeling process, you may fit a model on one dataset, then evaluate the model based on another. This might be a hold-out portion or new data that is observed later. 
- When applying transformations like center and standardize, you should be careful when using the model to form predications based on new data.
- These are called stateful transformations, because you must use statistics like the mean or standard deviation of the original dataset when transforming a new dataset.

In [26]:
# the patsy.build_design_matrices function can apply transformations to new out-of-sample data using the saved information from the original in-sample dataset

new_data = pd.DataFrame({
        'x0': [6, 7, 8, 9],
        'x1': [3.1, -0.5, 0, 2.3],
        'y': [1, 2, 3, 4]})
new_X = patsy.build_design_matrices([X.design_info], new_data)

new_X

[DesignMatrix with shape (4, 3)
   Intercept  x0  np.log(np.abs(x1) + 1)
           1   6                 1.41099
           1   7                 0.40547
           1   8                 0.00000
           1   9                 1.19392
   Terms:
     'Intercept' (column 0)
     'x0' (column 1)
     'np.log(np.abs(x1) + 1)' (column 2)]

In [27]:
# Because the plus symbol (+) in the context of Patsy formulas does not mean addition, when you want to add columns from a dataset by name, you must wrap them in the special I function

y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

# patsy has several other built-in transforms in the patsy.builtins module
# categorical data has a special class of transformations

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

## [ Categorical Data and Patsy ]
- Nonnumeric data can be transformed for a model design matrix in many different ways.
- A complete treatment of this topic will be very broad, and would be studied best along with a course in statistics.

- When you use nonnumeric terms in a Patsy formula, they are converted to dummy variables by default. If there is an intercept, one of the levels will be left out to avoid collinearity

In [28]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]})

y, X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [29]:
# if you omit the intercept from the model, then columns for each category value will be included in the model design matrix
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [30]:
# numeric columns can be interpreted as categorical with the C function
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [31]:
# when using multiple categorical terms in a model, things can be more complicated, as you can include interaction terms of the form key1:key2, which can be used, for example, in analysis of variance (ANOVA) methods:

data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [33]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [34]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

# Patsy provides for other ways to transform categorical data, including transformations for terms with a particular ordering

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)